## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [6]:
df =pd.read_csv( 'drive/MyDrive/Python for Data Analysis by Hanna Pylieva/Data/cookie_cats.csv')


In [10]:
df.groupby('version')['retention_7'].mean()

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

На основі цих результатів можна зробити припущення, що версія гри з воротами на 30 рівні має трохи краще(19.02%) утримання гравців через 7 днів після встановлення гри, порівняно з версією гри з воротами на 40 рівні (18.2%).

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [21]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [17]:
control_results = df[df['version'] == 'gate_30']['retention_7']
treatment_results = df[df['version'] == 'gate_40']['retention_7']

In [22]:
success = [control_results.sum(), treatment_results.sum()]
n_con = control_results.count()
n_treat = treatment_results.count()
nobs = [n_con, n_treat]

z_stat, p_value = proportions_ztest(success, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(success, nobs=nobs, alpha=0.05)

print(f'z stat: {z_stat: .4f}')
print(f'p-value: {p_value: .4f}')
print(f'Довірчий інтервал 95% для групи control:{lower_con: .4f}, {upper_con: .4f}')
print(f'Довірчий інтервал 95% для групи treatment:{lower_treat: .4f}, {upper_treat: .4f}')

z stat:  3.1644
p-value:  0.0016
Довірчий інтервал 95% для групи control: 0.1866,  0.1938
Довірчий інтервал 95% для групи treatment: 0.1785,  0.1855


1. Значення z-statistic дозволяє нам відхилити нульову гіпотезу про рівність середніх утримання на 7 день для двох груп. Тобто є статистично значуща різниця у поведінці користувачів між версіями гри gate_30 і gate_40.

2. Довірчі інтервали не перетинаються версіями гри gate_30 і gate_40. Це означає, що ми можемо мати впевненість на рівні довіри 95%, що середнє утримання користувачів на 7 день для групи gate_30 є вищим, ніж для групи gate_40.

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [23]:
from scipy.stats import chi2_contingency

In [24]:
contingency_table = pd.crosstab(df['version'], df['retention_7'])

contingency_table

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [25]:
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

print(f"Хі-квадрат статистика: {chi2_stat:.4f}")
print(f"p-value: {p_val:.4f}")

Хі-квадрат статистика: 9.9591
p-value: 0.0016


Отримане p-value (0.0016) менше рівня значущості 0.05. Це означає, що на рівні значущості 0.05 є достатні докази для відхилення нульової гіпотези (H0), що значення retention_7 не залежить від версії гри.
В такому разі приймається альтернативна гіпотеза (H1), що існує статистично значуща залежність між версією гри і значенням retention_7 (чи гравець повернувся на 7-й день після встановлення гри). Таким чином, можна вважати, що версія гри впливає на утримання користувачів на 7-й день після встановлення, та цей вплив є статистично значущим.